In [1]:
import requests
import json
from datetime import datetime
from scrapy.http import HtmlResponse
import hashlib

In [2]:
curr_time = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")

def get_hash(n):

    return hashlib.sha256(((n+"Dubai Financial Services Authority Delisted Securities, UAE" + "ARE_E20368").lower()).encode()).hexdigest()

In [10]:
p = 1
data = []
while p<10: 
    url = f'https://www.dfsa.ae/what-we-do/dfsa-listing-authority/delisted-securities?&ccm_paging_p={p}'

    r = requests.get(url)

    resp = HtmlResponse('example.com', body = r.text, encoding = 'utf-8')

    for a in resp.xpath('//div/div/div[2]/a'):
        data_dict={}
        name = a.xpath('./div[1]/p/text()').get()
        name = name.split('(')
        name = name[0].strip()
        security = a.xpath('./div[2]/p/text()').get().strip()
        exchange = a.xpath('./div[3]/p/text()').get().strip()
        isin = a.xpath('./div[4]/p/text()').get()
        terminate_date = a.xpath('./div[5]/p/text()').get()
        data_dict['uid'] = get_hash(name)
        data_dict['name'] = name
        data_dict['alias_name'] = []
        data_dict['list_type'] = 'Entity'
        data_dict['last_updated'] = curr_time
        data_dict['nns_status'] = 'False'
        data_dict['sanction_details'] = {'termination_date':terminate_date}
        data_dict['entity_details'] = {'security_type':security,'exchange_name':exchange}
        data_dict['country']=[]
        data_dict['address'] = [{'complete_addrss':'','country':''}]
        data_dict['documents'] = {'ISIN': [isin.strip()]}
        data_dict['comment'] = ''
        data_dict["sanction_list"]= {
            "sl_authority": "Dubai Financial Services Authority, UAE",
            "sl_url": "https://www.dfsa.ae/what-we-do/dfsa-listing-authority/delisted-securities",
            "sl_host_country": "UAE",
            "sl_type": "Sanctions",
            "watch_list": "EMEA Watchlists",
            "sl_source": "Dubai Financial Services Authority Delisted Securities, UAE",
            "sl_description": "List of securities delisted from its Exchange by Dubai Financial Services Authority, UAE",
            "list_id": "ARE_E20368"

        }
        data.append(data_dict)
    p+=1

In [11]:
with open('result_sng931.json', 'w', encoding="utf-8") as file:
    json.dump(data, file, ensure_ascii=False, indent=2, default=str)